# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,11.88,93,44,10.69,RU,1728599230
1,1,edinburgh of the seven seas,-37.0676,-12.3116,55.15,76,38,28.72,SH,1728599231
2,2,grytviken,-54.2811,-36.5092,30.09,95,100,5.21,GS,1728599232
3,3,prainha,-1.8000,-53.4800,87.55,43,56,4.34,BR,1728599233
4,4,hermanus,-34.4187,19.2345,58.39,78,99,5.53,ZA,1728599234


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humitidy_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles = 'OSM', size='Humidity', scale = 0.5, color='City') 

# Display the map
city_humitidy_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp']>60)&(city_data_df['Max Temp']<80)&(city_data_df['Humidity']<70)]

# Drop any rows with null values
dropped_df = ideal_cities_df.dropna()

# Display sample data
dropped_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,ararat,39.8317,44.7049,70.72,26,100,5.84,AM,1728599246
29,29,timimoun,29.2639,0.2310,77.38,12,0,0.00,DZ,1728599263
35,35,karratha,-20.7377,116.8463,78.28,51,4,16.44,AU,1728599272
36,36,metlili chaamba,32.2667,3.6333,79.74,28,0,5.75,DZ,1728599273
44,44,eureka,40.8021,-124.1637,66.25,64,35,4.00,US,1728599286
...,...,...,...,...,...,...,...,...,...,...
570,570,orchard homes,46.8633,-114.0484,69.60,42,0,4.61,US,1728599936
572,572,balkanabat,39.5108,54.3671,68.00,54,34,2.89,TM,1728599940
577,577,constantia,44.1833,28.6500,61.70,64,74,4.25,RO,1728599946
582,582,dawesville,30.9224,-84.0091,77.41,36,0,9.22,US,1728599952


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dropped_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
14,ararat,AM,39.8317,44.7049,26,
29,timimoun,DZ,29.2639,0.2310,12,
35,karratha,AU,-20.7377,116.8463,51,
36,metlili chaamba,DZ,32.2667,3.6333,28,
44,eureka,US,40.8021,-124.1637,64,
...,...,...,...,...,...,...
570,orchard homes,US,46.8633,-114.0484,42,
572,balkanabat,TM,39.5108,54.3671,54,
577,constantia,RO,44.1833,28.6500,64,
582,dawesville,US,30.9224,-84.0091,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
#for sha to find the difference in code between the two
radius = 32808
latitude = []
longitude = []
params = {'apiKey': geoapify_key, 'categories': 'accommodation.hotel'}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    filters = [f'circle:{longitude}, {latitude}. {radius}']
    bias = [f' proximity: {longitude}, {latitude}']
    params['categories'] = f'accommodation.hotel'
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    name_address = name_address.json()

    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ararat - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
eureka - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
kalabo - nearest hotel: No hotel found
kasempa - nearest hotel: No hotel found
queenstown - nearest hotel: No hotel found
susanville - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
long beach - nearest hotel: No hotel found
zhangatas - nearest hotel: No hotel found
baker city - nearest hotel: No hotel found
richmond - nearest hotel: No hotel found
saurimo - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
creel - nearest hotel: No hotel found
ocean shores - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
14,ararat,AM,39.8317,44.7049,26,No hotel found
29,timimoun,DZ,29.2639,0.2310,12,No hotel found
35,karratha,AU,-20.7377,116.8463,51,No hotel found
36,metlili chaamba,DZ,32.2667,3.6333,28,No hotel found
44,eureka,US,40.8021,-124.1637,64,No hotel found
...,...,...,...,...,...,...
570,orchard homes,US,46.8633,-114.0484,42,No hotel found
572,balkanabat,TM,39.5108,54.3671,54,No hotel found
577,constantia,RO,44.1833,28.6500,64,No hotel found
582,dawesville,US,30.9224,-84.0091,36,No hotel found


In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ararat - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
karratha - nearest hotel: Karratha International Hotel
metlili chaamba - nearest hotel: No hotel found
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
hamilton - nearest hotel: North Vista Manor
bardai - nearest hotel: No hotel found
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
kasempa - nearest hotel: Kasmpa Council Guest House
queenstown - nearest hotel: Queens Hotel
susanville - nearest hotel: SureStay Plus Hotel by Best Western Susanville
nouadhibou - nearest hotel: Hotel valencia
al kharijah - nearest hotel: Kharga Hotel
long beach - nearest hotel: Westin
zhangatas - nearest hotel: Арман
baker city - nearest hotel: Geiser Grand Hotel
richmond - nearest hotel: No hotel found
saurimo - nearest hotel: Hotel Princesinha
geraldton - nearest hotel: Ocean Centre Hotel
avarua - nearest hotel: Paradise Inn
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
t

,City,Country,Lat,Lng,Humidity,Hotel Name
14,ararat,AM,39.8317,44.7049,26,No hotel found
29,timimoun,DZ,29.2639,0.2310,12,أناف
35,karratha,AU,-20.7377,116.8463,51,Karratha International Hotel
36,metlili chaamba,DZ,32.2667,3.6333,28,No hotel found
44,eureka,US,40.8021,-124.1637,64,"Eureka Inn, Trademark Collection by Wyndham"
...,...,...,...,...,...,...
570,orchard homes,US,46.8633,-114.0484,42,Broadway Inn
572,balkanabat,TM,39.5108,54.3671,54,Nebitçi
577,constantia,RO,44.1833,28.6500,64,Hotel Dali
582,dawesville,US,30.9224,-84.0091,36,Baymont Thomasville


In [ ]:
### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_city_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size = 'Humidity',
    scale = 5,
    color = 'Hotel name',
    hover_cols = ['Hotel name', 'Country'],
    frame_width = 950,
    frame_height = 550
)    
    
# Display the map
hotel_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country)